# FORESEE - ALP with photon couplings

### Load Libraries 

In [ ]:
import numpy as np
import sys
import os

src_path = "../../../../src"
sys.path.append(src_path)
from foresee import Foresee, Utility, Model

from main import sigma_gNucleus_G2Nucleus_analyt_gammagamma, sigma_G2Nucleus_gNucleus_analyt_gammagamma
from constants import *
from utilities import get_t_min_t_max_1

from timeit import default_timer as timer

## 1. Initialization 

In [ ]:
foresee = Foresee(path=src_path)

We use the Primakoff cross section given by
\begin{equation}
    \sigma_{\gamma N \to G N} \simeq \frac{\alpha_{em} g_{\gamma \gamma}^2 Z^2}{2} \left(\log \left(\frac{d}{1/a^2 - t_{max}}\right)-2\right) + \frac{\alpha_{em} g_{\gamma \gamma}^2 Z^2}{12 m_G^4} \left(\log \left(1/d\right)-1\right) 
\end{equation}
where $t_{max} = -m_a^4 / (4 E_\gamma^2)$

## 2. G2 Production via Primakoff Process

In [ ]:
def get_spectrum_G2(mass, generator="EPOSLHC", energy="14"):
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    #define quantities
    alpha, me, ZFe, AFe = 1./137., 0.000511, 26, 56
    a, d  = 111.*ZFe**(-1./3.) / me,  0.164 * AFe**(-2./3.)
    SMXSinBarn, GeV2inBarn = 5, 0.389379e-3

    SMXSinIGeV2 = 13311.9696379   # 5.2 barn - photon conversion cross-section on Fe
    
    # prepare output file
    filename_out = energy+"TeV/Prim_"+energy+"TeV_"+str(mass)+".txt"
    f = open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")
        
    # loop over photons in spectrum
    for logth,logen,wgt in photons:
        if wgt==0: 
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        k = 10**logen
        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_Fe
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)       # 
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_Fe, A=A_Fe, t_max=t_max)
        prob = xs / SMXSinIGeV2
        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    f.close()

num_of_masses = 31
masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-3,1,num_of_masses)]]
masses = np.array(masses)

for count, mass in enumerate(masses):
    start = timer()
    get_spectrum_G2(mass=mass, generator="EPOSLHC", energy="14")
    end = timer()
    time_length_sec = end - start

    count_total = masses.shape[0]
    count += 1
    time_length_sec_total = time_length_sec * count_total
    print("%.2f%% done, " % float(count / masses.shape[0] * 100))

## G2 Production via Primakoff Process - NOMAD

In [ ]:
def get_spectrum_G2_NOMAD(mass):
    # generator="NOMAD"
    generator="Pythia"
    energy="0.45"
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    SMXSinIGeV2 = 4.62274197928718e+2  # 0.18 barn - photon conversion cross-section on Be

    filename_out = energy+"TeV/NOMAD_Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")
        
    # loop over photons in spectrum
    for logth, logen, wgt in photons:
        if wgt==0: 
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        xs, xsx = 0, 0
        k = 10**logen
        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_Be
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_Be, A=A_Be, t_max=t_max)

        prob = xs / SMXSinIGeV2
        POT = 4.1e19
        prob *= POT

        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    f.close()


masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,np.log10(0.5),21)]]
for mass in masses: 
    get_spectrum_G2_NOMAD(mass=mass)

## G2 Production via Primakoff Process - SHiP


In [ ]:
def get_spectrum_G2_SHiP(mass):
    # generator="SHiP"
    generator="Pythia"
    energy="0.4"
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    SMXSinIGeV2 = 3.23591938550102e+4  # 12.6 barn - photon conversion cross-section on Mo
    
    filename_out = energy+"TeV/SHiP_Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")
        
    # loop over photons in spectrum
    for logth, logen, wgt in photons:
        if wgt==0: 
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        xs, xsx = 0, 0
        k = 10**logen
        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_Mo
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)       # 
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_Mo, A=A_Mo, t_max=t_max)

        prob = xs / SMXSinIGeV2
        POT = 2e20
        prob *= POT

        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    f.close(


masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-3,0.5,51)]]
for mass in masses: 
    get_spectrum_G2_SHiP(mass=mass)

## G2 Production via Primakoff Process - CHARM

In [ ]:
def get_spectrum_G2_CHARM(mass):
    # generator="CHARM"
    generator="Pythia"
    energy="0.4"
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    SMXSinIGeV2 = 1.64364159263544e+4  # 6.4 barn - photon conversion cross-section on Cu
    
    filename_out = energy+"TeV/CHARM_Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")
        
    # loop over photons in spectrum
    for logth, logen, wgt in photons:
        if wgt==0: 
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        xs, xsx = 0, 0
        k = 10**logen
        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_Cu
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_Cu, A=A_Cu, t_max=t_max)

        prob = xs / SMXSinIGeV2
        POT = 2.4e18
        prob *= POT

        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    
    f.close()

masses_CHARM = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-3,-0.5,21)]]
masses_CHARM = np.array(masses_CHARM)

for count, mass in enumerate(masses_CHARM):
    start = timer()
    get_spectrum_G2_CHARM(mass=mass)
    end = timer()
    time_length_sec = end - start

    count_total = masses_CHARM.shape[0]
    count += 1
    time_length_sec_total = time_length_sec * count_total
    print("%.2f%% done, " % float(count / masses_CHARM.shape[0] * 100))

## G2 Production via Primakoff Process - NA62

In [ ]:
def get_spectrum_G2_NA62(mass):
    # generator="NA62"
    generator="Pythia"
    energy="0.4"
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    SMXSinIGeV2 = 1.64364159263544e+4  # 6.4 barn - photon conversion cross-section on Cu
    
    filename_out = energy+"TeV/NA62_Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")
        
    # loop over photons in spectrum
    for logth, logen, wgt in photons:
        if wgt==0: 
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        xs, xsx = 0, 0
        k = 10**logen
        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_Cu
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_Cu, A=A_Cu, t_max=t_max)

        prob = xs / SMXSinIGeV2
        POT = 1e18
        prob *= POT

        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    f.close()

masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,np.log10(1),21)]]
for mass in masses: 
    get_spectrum_G2_NA62(mass=mass)

## G2 Production via Primakoff Process - SeaQuest

In [ ]:
def get_spectrum_G2_SeaQuest(mass):
    generator="Pythia"
    energy="0.12"
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)

    SMXSinIGeV2 = 13311.9696379   # 5.2 barn - photon conversion cross-section on Fe
    
    filename_out = energy+"TeV/SeaQuest_Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")

    # loop over photons in spectrum
    for logth, logen, wgt in photons:
        if wgt==0:
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        xs, xsx = 0, 0
        k = 10**logen
        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_Fe
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_Fe, A=A_Fe, t_max=t_max)

        prob = xs / SMXSinIGeV2
        POT = 1.44e18
        prob *= POT

        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    f.close()

masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,0,21+1)]]
for mass in masses: 
    get_spectrum_G2_SeaQuest(mass=mass)

## G2 Production via Primakoff Process - DUNE

In [ ]:
def get_spectrum_G2_DUNE(mass):
    generator="Pythia"
    energy="0.12"
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    SMXSinIGeV2 = 933.0409190219502  # photon conversion cross-section on C (graphite)
    
    filename_out = energy+"TeV/DUNE_Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")

    # loop over photons in spectrum
    for logth, logen, wgt in photons:
        if wgt==0:
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        xs, xsx = 0, 0
        k = 10**logen

        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_C
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)       # t_min < t_max < 0
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_C, A=A_C, t_max=t_max)

        prob = xs / SMXSinIGeV2
        POT = 1e22
        prob *= POT

        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")

    f.close()


masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,0,21+1)]]
for mass in masses: 
    get_spectrum_G2_DUNE(mass=mass)

## G2 Production via Primakoff Process - NuCal

In [ ]:
def get_spectrum_G2_NuCal(mass):
    # generator="NuCal"
    generator="Pythia"
    energy="0.069"
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    SMXSinIGeV2 = 13311.9696379   # 5.2 barn - photon conversion cross-section on Fe

    filename_out = energy+"TeV/NuCal_Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")
        
    # loop over photons in spectrum
    for logth, logen, wgt in photons:
        if wgt==0: 
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        xs, xsx = 0, 0
        k = 10**logen
        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_Fe
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_Fe, A=A_Fe, t_max=t_max)

        prob = xs / SMXSinIGeV2
        POT = 1.7e18
        prob *= POT

        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    f.close()

masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,np.log10(0.5),21)]]
for mass in masses: 
    get_spectrum_G2_NuCal(mass=mass)

## G2 Production via Primakoff Process - PS191

In [ ]:
def get_spectrum_G2_PS191(mass):
#     generator="PS191"
    generator="Pythia"
    energy="0.0192"
    
    #load photon spectrum
    filepath = "../../../../" + "files/hadrons/"+energy+"TeV/"+generator+"/"+generator+"_"+energy+"TeV_22.txt"
    photons = foresee.readfile(filepath)
    
    SMXSinIGeV2 = 4.62274197928718e+2  # 0.18 barn - photon conversion cross-section on Be

    filename_out = energy+"TeV/PS191_Prim_"+energy+"TeV_"+str(mass)+".txt"
    f= open(filename_out,"w+")
    f.write("# This file contains the cross section for ALP-photon with g=1 into the FORWARD hemisphere only\n")
    f.write("#  log10(theta)    log10(p/GeV)     cross-section[pb/bin]\n")
        
    # loop over photons in spectrum
    for logth, logen, wgt in photons:
        if wgt==0: 
            f.write(str(logth)+" "+str(logen)+" "+str(wgt)+"\n")
            continue

        xs, xsx = 0, 0
        k = 10**logen
        m1, m3 = 0.0, mass
        m2 = m4 = M_NUCLEUS_Be
        s = m1**2 + 2 * k * m2 + m2**2

        t_min, t_max = get_t_min_t_max_1(s, m1, m2, m3, m4)       # t_min < t_max < 0
        xs = sigma_gNucleus_G2Nucleus_analyt_gammagamma(g_gg=1, s=s, m1=0.0, m3=mass, Z=Z_Be, A=A_Be, t_max=t_max)

        prob = xs / SMXSinIGeV2
        POT = 8.6e18
        prob *= POT

        if np.isnan(prob): prob=0
        f.write(str(logth)+" "+str(logen)+" "+str(wgt*prob)+"\n")
    
    f.close()

masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,np.log10(0.5),21)]]
for mass in masses: 
    get_spectrum_G2_PS191(mass=mass)